In [1]:
import urllib.request
import os.path
import numpy as np
import sys
sys.path.insert(0, './')
from utils import plot_mats

In [17]:
class ImageSplitter:
    """ splits the moving mnist dataset into shorts of n frames
    
    Moving Mnist: 
        10.000 sequences of length 20 showing 2 digits moving in 64x64
    
    """
    def __init__(self, 
                 n=3, 
                 ntest=1000,
                 file_name = 'mnist_test_seq.npy',
                 url = 'http://www.cs.toronto.edu/~nitish/unsupervised_video/mnist_test_seq.npy',
                 batchsize=32):
        """ ctor
            n: integer, length of subsequence
            ntest: integer, number of items that belong to the test set
            file_name: string, name used to safe the data locally
            url: url from where data can be downloaded
            batchsize: ~
            
        """
        
        self.batchsize = batchsize
        self.batch_loop = 0
        self.n = n
        
        if not os.path.isfile(file_name):
            print("could not find dataset: download it..")
            urllib.request.urlretrieve(url, file_name)
            print("download complete")
    
        moving_mnist = np.load(file_name) # shape: 20,10000,64,64
        
        moving_mnist = np.rollaxis(moving_mnist, 1) # --> 10000,20,64,64
        np.random.shuffle(moving_mnist)
        
        self.test_set = moving_mnist[0:ntest]
        self.train_set = moving_mnist[ntest:]
        
        F, N, H, W = moving_mnist.shape # Frames, Numbers, Height, Width
        
        
        self.vector_dimension = H * W
        
    def get_dimension(self):
        """ gets the data dimension
        """
        return self.vector_dimension

    def transform_to_n_gram(self, batch):
        """ transforms the batch into an n-gram (parameter n)
        batch: np.array((batchsize, 20, 64, 64))
        """
        n = self.n
        N, M, H, W = batch.shape
        seqs = M-n+1 # sequences per video
        
        Result = np.zeros((seqs * N, n, 64, 64), 'uint8')
        
        pos = 0
        for j in range(N):
            for i in range(seqs):
                Result[pos] = batch[j, i:i+n]
                pos += 1 
        
        N, M, H, W = Result.shape
        return Result.reshape((N, M, H * W))
        
    
    def next_batch(self):
        """ returns the next batch
        """
        start = self.batch_loop
        end = self.batch_loop + self.batchsize
        N = self.train_set.shape[0]
        if N > end:
            self.batch_loop = end
            return self.transform_to_n_gram(
                self.train_set[start:end])
        else:
            diff = (N - start)
            end = self.batchsize - diff
            
            set1 = self.train_set[start:N]
            set2 = self.train_set[0:end]
            
            self.batch_loop = end
            return self.transform_to_n_gram(
                np.concatenate((set1, set2)))
        

splitter = ImageSplitter()

out = splitter.next_batch()
print(out.shape)

(576, 3, 4096)


In [20]:
import tensorflow as tf

# Util
def random_binomial(shape, p=0.0, dtype=None, seed=None):
    """Returns a tensor with random binomial distribution of values.
    # Arguments
        shape: A tuple of integers, the shape of tensor to create.
        p: A float, `0. <= p <= 1`, probability of binomial distribution.
        dtype: String, dtype of returned tensor.
        seed: Integer, random seed.
    # Returns
        A tensor.
    """
    if dtype is None:
        dtype = 'float32'
    if seed is None:
        seed = np.random.randint(10e6)
    return tf.where(tf.random_uniform(shape, dtype=dtype, seed=seed) <= p,
                    tf.ones(shape, dtype=dtype),
                    tf.zeros(shape, dtype=dtype))



numpy_rng = np.random.RandomState(1)

dim = splitter.get_dimension()
numfac  = 512
nummap  = 256

# pretrainig first layer
input_x1 = tf.placeholder(tf.float32, [None, dim])
input_x2 = tf.placeholder(tf.float32, [None, dim])

U1 = tf.Variable(tf.random_normal(shape=(dim, numfac)) * 0.01)
V1 = tf.Variable(tf.random_normal(shape=(dim, numfac)) * 0.01)
W1 = tf.Variable(numpy_rng.uniform(low=-0.01, high=+0.01, size=( numfac, nummap)).astype('float32'))

bias_U1 = tf.Variable(np.zeros(numfac, dtype='float32'))
bias_V1 = tf.Variable(np.zeros(numfac, dtype='float32'))
bias_W1 = tf.Variable(np.zeros(nummap, dtype='float32'))
bias_U1_out = tf.Variable(np.zeros(dim, dtype='float32'))
bias_V1_out = tf.Variable(np.zeros(dim, dtype='float32'))
bias_W1_out = tf.Variable(np.zeros(numfac, dtype='float32'))

# m = sigmoid(W(U*x_1 . V*x_2 ))
M1 =  tf.sigmoid(tf.matmul(tf.multiply(tf.matmul(input_x1,U1) + bias_U1,tf.matmul(input_x2,V1) + bias_V1), W1)+ bias_W1)

output_x1 = tf.matmul(tf.multiply(tf.matmul(M1,tf.transpose(W1)) + bias_W1_out,tf.matmul(input_x2,V1) + bias_V1),tf.transpose(U1))+ bias_U1_out
output_x2 = tf.matmul(tf.multiply(tf.matmul(M1,tf.transpose(W1)) + bias_W1_out,tf.matmul(input_x1,U1) + bias_U1), tf.transpose(V1))+ bias_V1_out

cost_1 = tf.nn.l2_loss(output_x1-input_x1) + tf.nn.l2_loss(output_x2-input_x2)
optimizer_1 = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost_1)

print('xx')

#class PredictiveGatingPyramid:
#    """
#    """
#    
#    def __init__(self):
#        pass
#pgp = PredictiveGatingPyramid()

xx
